## SILVER TO GOLD LAYER


In [1]:
import pandas as pd
from common_utilities import Portfolio,global_paths

### Gold Layer - Trade History


In [2]:
# read the csv file
df_TradeHistory = pd.read_csv(TradeHistory_SILVER_FILE_PATH)

# replace scrip code to compnay name
df_Symbol = pd.read_csv(Symbol_SILVER_FILE_PATH)
df_Symbol["scrip_code"] = df_Symbol["scrip_code"].astype(str)

# Convert 'datetime' to datetime type
df_TradeHistory["datetime"] = pd.to_datetime(df_TradeHistory["datetime"])

# Merge df_TradeHistory with df_Symbol on the matching columns
df_TradeHistory = df_TradeHistory.merge(
    df_Symbol[["scrip_code", "symbol"]],
    left_on="scrip_code",
    right_on="scrip_code",
    how="left",
)

# Assign the new column 'stock_name' in df_TradeHistory to the values from 'symbol'
df_TradeHistory["stock_name"] = df_TradeHistory["symbol"].combine_first(df_TradeHistory["stock_name"])

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by="datetime")


portfolio = Portfolio()
data = [portfolio.trade(row) for _, row in df_TradeHistory.iterrows()]

df_TradeHistory = pd.DataFrame(data)
# df_TradeHistory = df_TradeHistory.round(2)
df_TradeHistory.columns()
df_TradeHistory = df_TradeHistory[
    [
        "datetime",
        "segment",
        "stock_name",
        "side",
        "buy_quantity",
        "buy_price",
        "buy_amount",
        "sell_quantity",
        "sell_price",
        "sell_amount",
        "profit_loss_amount",
        "holding_quantity",
        "holding_price_avg",
        "holding_amount",
    ]
]

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by=["segment", "stock_name", "datetime"])

# Save the result as a csv file
df_TradeHistory.to_csv(TradeHistory_GOLD_FILE_PATH, index=None)
df_TradeHistory.info()
print("GOLD Layer csv file for trade history successfully created at:")
print(TradeHistory_GOLD_FILE_PATH.resolve())